In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
shakespeare_url = "https://homl.info/shakespeare" # shortcut URL 
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url) 
with open(filepath) as f:
    shakespeare_text = f.read()

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
# convert characters to numbers - Tokenizer class
# char_level = True to get character level encoding
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespeare_text])

In [4]:
import numpy as np

In [5]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [6]:
max_id = len(tokenizer.word_index)
dataset_size = len(shakespeare_text)

In [7]:
# spliting training data - 90 % 
# Using the tf data set API to get one character at a time from the tensor.

train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [8]:
# Window method
# by default it is nonoverlapping, using shift =1 gives more training data. 
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead 
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [9]:
# Using flat_map to convert the datasets to tensorflow.
# we can pass any function in the flat_map to perform transformation as well.

dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [10]:
# Shuffling the data - Since GD will work better. 

batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [11]:
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [12]:
# Create a Char RNN model
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
                     #dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True),
                     #dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                activation="softmax"))])

In [13]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

history = model.fit(dataset, epochs=5)

Epoch 1/5
31368/31368 [==============================] - 1660s 53ms/step - loss: 0.9598
Epoch 2/5
31368/31368 [==============================] - 1668s 53ms/step - loss: 0.9595
Epoch 3/5
31368/31368 [==============================] - 1669s 53ms/step - loss: 1.0185
Epoch 4/5
31368/31368 [==============================] - 1668s 53ms/step - loss: 1.0686
Epoch 5/5
31368/31368 [==============================] - 1669s 53ms/step - loss: 1.1085


In [14]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1 
    return tf.one_hot(X, max_id)

In [15]:
X_new = preprocess(["How are yo"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


'u'

In [16]:
# Generating Fake Shakespearean Text

def next_char(text, temperature=1):
  X_new = preprocess([text])
  y_proba = model.predict(X_new)[0, -1:, :]
  rescaled_logits = tf.math.log(y_proba) / temperature
  char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1 
  return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [17]:
def complete_text(text, n_chars=50, temperature=1): 
  for _ in range(n_chars):
    text += next_char(text, temperature) 
  return text

In [19]:
 print(complete_text("t", temperature=0.2))

thereforachaming whom to the house of me as for my 


In [21]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [22]:
print(complete_text("w", temperature=1))

whoms workhoo, to for me and him, hold patron
it: h
